In [ ]:
# !pip install yfinance
# !pip install scipy

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata
tickers_list = ['VNQ', 'EEM', 'EFA', 'DBC', 'SPY', 'TLT']
number_of_tickers = len(tickers_list)

In [ ]:
import yfinance as yf
start_date = '2022-04-30'
adj_close_price = yf.download(tickers_list,start_date, auto_adjust=False)['Adj Close'] # 2025-02-27: yf API changed. The default auto_adjust=True gives only adjusted OHLC, not giving AdjClose, so impossible to reverse engineer the splits, dividindends and rawPrices. The auto_adjust=false gives OHLC (raw) + 'Adj Close'.
print(adj_close_price)

In [ ]:
pct_changes = adj_close_price.pct_change()
print(pct_changes)

In [ ]:
total_return = np.divide(adj_close_price.iloc[-1],adj_close_price.iloc[0]) - 1
print(total_return)

In [ ]:
total_return_order = number_of_tickers - rankdata(total_return,method='average') + 1
print(total_return_order)

In [ ]:
total_sd = np.std(pct_changes)
print(total_sd)

In [ ]:
total_sd_order = rankdata(total_sd,method='average')
print(total_sd_order)

In [ ]:
corr_mtx = pct_changes.corr()
total_corr = np.average(corr_mtx, axis=1)
print(corr_mtx)
print(total_corr)

In [ ]:
total_corr_order = rankdata(total_corr,method='average')
print(total_corr_order)

In [ ]:
sub_strat_weigths = [0.5, 0.25, 0.25]
print(sub_strat_weigths)

In [ ]:
rank_mtx = np.asmatrix([total_return_order, total_sd_order, total_corr_order])
print(rank_mtx)

In [ ]:
total_score = np.matmul(sub_strat_weigths,rank_mtx)
print(total_score)

In [ ]:
total_score_order = rankdata(total_score,method='min')
print(total_score_order)

In [ ]:
number_of_selected_tickers = 4
selected_tickers = (total_score_order < number_of_selected_tickers).astype(int)
print(selected_tickers)

In [ ]:
absolute_momentum_filter = (total_return > 0).astype(int)
print(absolute_momentum_filter)


In [ ]:
final_signals = np.multiply(selected_tickers, absolute_momentum_filter)
print(final_signals)